In [1]:
import pandas as pd
import numpy as np
import sys
import time as tm
from allocation_solution_df import *
from vaccine_alloc_instance_df import *
from solvers.offline_maxutility_df import *
from solvers.online_maxutility_df import *

In [3]:
#importing agents data
agents_type_df = pd.read_pickle('data/agents_type.pkl')
availability_df = pd.read_pickle('data/availability_df.pkl')
belongingness_df = pd.read_pickle('data/belongingness_df.pkl')

#importing hospitals data
daily_supply_df = pd.read_pickle('data/daily_supply.pkl')
overall_quota_df = pd.read_pickle('data/overall_quota.pkl')
Hospital_Data_Chennai_May_2022_df = pd.read_pickle('data/Hospital_Data_Chennai_May_2022.pkl')

In [4]:
daily_supply_df

,Date,Total_available_capacity_per_day
0,01-05-2022,147
1,02-05-2022,189
2,03-05-2022,178
3,04-05-2022,214
4,05-05-2022,80
5,06-05-2022,537
6,07-05-2022,495
7,08-05-2022,109
8,09-05-2022,170
9,10-05-2022,220


In [5]:
distances_df = pd.read_pickle('data/Distance_Data.pkl')

In [19]:
number_zip = len(dict.fromkeys(Hospital_Data_Chennai_May_2022_df['Zip_Code'].values.tolist()))
number_zip

19

In [22]:
fake_zips = [100001+i for i in range(number_zip)]

In [23]:
anon_zip = dict(zip(list(dict.fromkeys(Hospital_Data_Chennai_May_2022_df['Zip_Code'].values.tolist())),fake_zips))

In [24]:
anon_zip

{600034: 100001,
 600089: 100002,
 600116: 100003,
 600037: 100004,
 600085: 100005,
 600096: 100006,
 600024: 100007,
 600006: 100008,
 600035: 100009,
 600083: 100010,
 600020: 100011,
 600032: 100012,
 600004: 100013,
 600018: 100014,
 600028: 100015,
 600086: 100016,
 600106: 100017,
 600102: 100018,
 600093: 100019}

In [25]:
anon_distances_df=distances_df.replace({"Source": anon_zip, "Destination": anon_zip})

In [26]:
pd.set_option('display.max_rows', 500)
anon_distances_df

,Source,Destination,Distance
0,100013,100008,4251
1,100013,100014,2206
2,100013,100011,4867
3,100013,100007,6399
4,100013,100015,2442
5,100013,100012,10993
6,100013,100001,5633
7,100013,100009,3715
8,100013,100004,14994
9,100013,100010,9641


In [27]:
anon_distances_df.to_pickle('data-anonymized/Distance_Data.pkl')

In [28]:
centers = Hospital_Data_Chennai_May_2022_df['Center_ID'].values.tolist()

In [29]:
number_center = len(dict.fromkeys(centers))

In [30]:
fake_centers = [12301+i for i in range(number_center)]

In [31]:
fake_centers_dict = dict(zip(list(dict.fromkeys(centers)),fake_centers))

In [60]:
hosp_names = Hospital_Data_Chennai_May_2022_df['Center_Name'].values.tolist()

In [61]:
number_hosp_names = len(dict.fromkeys(hosp_names))

In [64]:
fake_hosp_names = ["Hospital_"+str(i+1) for i in range(number_hosp_names)]

In [66]:
fake_hosp_dict = dict(zip(list(dict.fromkeys(hosp_names)),fake_hosp_names))

In [72]:
anon_hosp_df = Hospital_Data_Chennai_May_2022_df.replace({"Zip_Code": anon_zip, "Center_ID": fake_centers_dict,
                                                         "Center_Name": fake_hosp_dict})

In [74]:
Hospital_Data_Chennai_May_2022_df

,Date,Zip_Code,Center_ID,Center_Name,Total_available_capacity
0,01-05-2022,600034,603898,Kanchi Kamakoti Child Trust,9
1,01-05-2022,600089,617607,Srushti Hospital Pvt Ltd,86
3,01-05-2022,600116,855907,Srushti Clinic Porur,52
5,02-05-2022,600037,604136,MMRV Hospital,19
6,02-05-2022,600085,1037942,Srushti Hospital Kottupuram,29
...,...,...,...,...,...
162,30-05-2022,600089,617607,Srushti Hospital Pvt Ltd,58
164,30-05-2022,600093,580753,Sooriya Hospital,74
165,31-05-2022,600024,580742,Medway Hospitals,59
166,31-05-2022,600089,617607,Srushti Hospital Pvt Ltd,59


In [77]:
anon_hosp_df.to_pickle('data-anonymized/hospital_data.pkl')

In [80]:
new_overall_quota_df = overall_quota_df.replace({"Center_ID": fake_centers_dict, "Center_Name": fake_hosp_dict})

In [82]:
new_overall_quota_df.to_pickle('data-anonymized/overall_quota_df.pkl')

In [ ]:
new_belongingness_df =  belongingness_df.rename(columns=fake_centers_dict)
new_belongingness_df.to_pickle('data-anonymized//belongingness_df.pkl')

In [17]:
len(dict.fromkeys(Hospital_Data_Chennai_May_2022_df['Zip_Code'].values.tolist()))

19